In [4]:
#同时输出一个cell多个结果
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all' 
import numpy as np
import pandas as pd
from sklearn.model_selection  import cross_val_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# 读入数据

In [8]:
iris=pd.read_csv('/Users/tianyc/Desktop/iris.csv')  #读入iris数据集，150个样本的5个特征 sepallength（花萼长度）petallength（花瓣长度），三个类别
#根据特征分为两部分，分别为预测特征target 和数据特征集 data
iris.head(20)


,Unnamed: 0,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,1,5.1,3.5,1.4,0.2,setosa
1,2,4.9,3.0,1.4,0.2,setosa
2,3,4.7,3.2,1.3,0.2,setosa
3,4,4.6,3.1,1.5,0.2,setosa
4,5,5.0,3.6,1.4,0.2,setosa
5,6,5.4,3.9,1.7,0.4,setosa
6,7,4.6,3.4,1.4,0.3,setosa
7,8,5.0,3.4,1.5,0.2,setosa
8,9,4.4,2.9,1.4,0.2,setosa
9,10,4.9,3.1,1.5,0.1,setosa



# 预处理

In [9]:
iris_data_s= pd.get_dummies(iris['Species'])   #species 类别onehot编码


columns_n=['Sepal.Length','Sepal.Width','Petal.Length'] #选三个数值型
iris_data_n_norm=MinMaxScaler().fit_transform(iris[columns_n])# 数值型0-1规范化

# 上一步规范化输出为ndarray类型，这里转换为DataFrame，并增加列索引
iris_data_n=pd.DataFrame(iris_data_n_norm,columns=('Sepal.Length','Sepal.Width','Petal.Length'))

#两组数据组合到一起，用concat函数，axis=1表示纵向连接
iris_data= pd.concat([iris_data_s,iris_data_n],axis=1)  
iris_target=iris['Petal.Width'] #预测花瓣宽度
iris_data  # 用于预测的特征

,setosa,versicolor,virginica,Sepal.Length,Sepal.Width,Petal.Length
0,1,0,0,0.222222,0.625000,0.067797
1,1,0,0,0.166667,0.416667,0.067797
2,1,0,0,0.111111,0.500000,0.050847
3,1,0,0,0.083333,0.458333,0.084746
4,1,0,0,0.194444,0.666667,0.067797
...,...,...,...,...,...,...
145,0,0,1,0.666667,0.416667,0.711864
146,0,0,1,0.555556,0.208333,0.677966
147,0,0,1,0.611111,0.416667,0.711864
148,0,0,1,0.527778,0.583333,0.745763


In [4]:
X =iris_data
y=iris_target
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)#留出法划分数据集

# 回归模型练习

## 线性回归

<p><font size="3" color="red"> 多元线性回归（最小二乘法） <br/>
 &nbsp;&nbsp; LinearRegression（）参数：<br/>
 &nbsp;&nbsp;-copy_X:默认为True，X会被复制,False将改变原数据;</p>

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
lr = LinearRegression(copy_X=False)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print (lr.coef_)#训练系数coef_为w1到w6
print (lr.intercept_)#截距intercept_为w0
# MSE
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
#R^2
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))

LinearRegression(copy_X=False)

[-0.49553687  0.07436052  0.42117636 -0.32457703  0.49282563  1.53893647]
0.38577504154186815
Mean squared error: 0.03
Coefficient of determination: 0.95


In [6]:
#交叉验证
from sklearn.model_selection import cross_val_predict
predicted = cross_val_predict(lr, X, y, cv=10) #cross_val_predict返回回归结果,每行一个预测值
print ( "MSE: %.2f" %mean_squared_error(y, predicted)) #MSE
print ( "R^2: %.2f" %r2_score(y, predicted)) #R^2


MSE: 0.03
R^2: 0.94


## 多项式回归

<p><font size="3" color="red"> 首先构造多项式特征，再用线性回归拟合<br/>
    PolynomialFeatures()参数：<br/>
 &nbsp;&nbsp;  
degree：控制多项式的度；
    <br/> &nbsp;&nbsp;include_bias:默认为True，包含常数项</p>

In [7]:
from sklearn.preprocessing import PolynomialFeatures
pr = PolynomialFeatures(degree=2, include_bias=False)
pr.fit(X, y)  
X2=pr.transform(X)   #构造包含2次项的特征
X2_train,X2_test,y2_train,y2_test = train_test_split(X2,y,test_size=0.3)#留出法划分数据集
lr.fit(X2_train, y2_train) #线性回归拟合
y_pred = lr.predict(X2_test) #预测值

print (lr.coef_)#训练系数coef_为w1到w6
print (lr.intercept_)#截距intercept_为w0
# MSE
print('Mean squared error: %.2f' % mean_squared_error(y2_test, y_pred))
#R^2
print('Coefficient of determination: %.2f' % r2_score(y2_test, y_pred))

PolynomialFeatures(include_bias=False)

LinearRegression(copy_X=False)

[-2.26655760e-01 -1.38502511e-01  3.65158271e-01  1.22131489e-01
  7.57605127e-01  4.67050195e-01 -2.26655760e-01  1.44328993e-15
  2.22044605e-16  1.34260483e+00 -1.67534557e+00  6.41604367e-01
 -1.38502511e-01 -2.77555756e-16 -5.01763404e-01  6.89904740e-01
  5.54785961e-01  3.65158271e-01 -7.18709933e-01  1.74304596e+00
 -7.29340133e-01  1.18709835e+00 -3.08446789e+00  2.10197082e-01
  1.46791752e+00 -6.70800884e-01  5.41391859e-01]
0.6823739768689185
Mean squared error: 0.03
Coefficient of determination: 0.95


## Lasso回归

<p><font size="3" color="red"> Lasso()参数：<br/>
 &nbsp;&nbsp;  
alpha : float, 正则化项的系数，默认 1.0；<br/>
 &nbsp;&nbsp; max_iter : int, 可选，最大循环次数<br/>
           
</p>

In [8]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.05)

lasso.fit(X_train, y_train)# 拟合训练集
y_pred = lasso.predict(X_test) #预测值
print(lasso.intercept_)
print(lasso.coef_)
print ( "MSE: %.2f" %mean_squared_error(y_test, y_pred)) #MSE
print ( "R^2: %.2f" %r2_score(y_test, y_pred)) #R^2

Lasso(alpha=0.05)

1.318192322849232
[-0.93793829 -0.          0.51563121  0.          0.          0.        ]
MSE: 0.07
R^2: 0.87


## 岭回归Ridge

<p><font size="3" color="red"> Ridge()参数：<br/>
 &nbsp;&nbsp;  
alpha : float, 正则化项的系数，默认 1.0；<br/>
 &nbsp;&nbsp; max_iter : int, 可选，最大循环次数<br/>
 &nbsp;&nbsp;  solver 求解方法solver='lsqr'：最小二乘法,solver='sag'：随机平均梯度下降法
</p>
 


In [9]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=1.0,solver='sag')
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test) #预测值
print(ridge.intercept_)
print(ridge.coef_)
print ( "MSE: %.2f" %mean_squared_error(y_test, y_pred)) #MSE
print ( "R^2: %.2f" %r2_score(y_test, y_pred)) #R^2

Ridge(solver='sag')

0.7334665943145537
[-0.6774565   0.10450665  0.57294985  0.21362232  0.19468946  0.58738996]
MSE: 0.04
R^2: 0.94


## 弹性网络 

<p><font size="3" color="red">  ElasticNet()参数：<br/>
 &nbsp;&nbsp; alpha : float, 正则化项的系数，默认 1.0；<br/>
 &nbsp;&nbsp; l1_ratio为L1范数惩罚项所占比例，l1_ratio[0,1] 若l1_ratio =0时，弹性网络退化为ridge（只剩L2范数的惩罚项）。<br/>
     &nbsp;&nbsp; max_iter : int, 可选，最大循环次数<br/>
        </p>

In [10]:
from sklearn.linear_model import ElasticNet
elasticNet = ElasticNet(alpha=0.1, l1_ratio=0.6)
elasticNet.fit(X_train, y_train)
y_pred = elasticNet.predict(X_test) #预测值
print(elasticNet.intercept_)
print(elasticNet.coef_)
print ( "MSE: %.2f" %mean_squared_error(y_test, y_pred)) #MSE
print ( "R^2: %.2f" %r2_score(y_test, y_pred)) #R^2

ElasticNet(alpha=0.1, l1_ratio=0.6)

1.2051970138533437
[-0.72743746  0.          0.43638767  0.         -0.          0.13881837]
MSE: 0.10
R^2: 0.82


## GBDT

 <p><font size="3" color="red">  GradientBoostingRegressor()参数：<br/>
 &nbsp;&nbsp; n_estimators: 最大的弱学习器的个数，默认100；<br/>
 &nbsp;&nbsp; learning_rate: 步长，默认1。<br/>
     &nbsp;&nbsp; subsample: 子采样（无放回），取值为(0,1]<br/>
    &nbsp;&nbsp; max_depth：决策树最大深度<br/>
   &nbsp;&nbsp;  min_samples_split：内部节点再划分所需最小样本数，默认2<br/>
     &nbsp;&nbsp; min_samples_leaf : 叶子节点最少样本数<br/></p>

In [11]:
from sklearn.ensemble import GradientBoostingRegressor
gbdt = GradientBoostingRegressor(random_state=10)
gbdt.fit(X_train, y_train)
y_pred = gbdt.predict(X_test) #预测值
gbdt.feature_importances_  #特征重要度
print ( "MSE: %.2f" %mean_squared_error(y_test, y_pred)) #MSE
print ( "R^2: %.2f" %r2_score(y_test, y_pred)) #R^2

GradientBoostingRegressor(random_state=10)

array([0.39047564, 0.04779821, 0.04292859, 0.01750569, 0.01766403,
       0.48362785])

MSE: 0.03
R^2: 0.94


## 随机梯度下降回归

In [12]:
from sklearn.linear_model import SGDRegressor
sgdr = SGDRegressor(loss='squared_error',penalty='l2',alpha=0.01,max_iter=1000)#squared error（平方误差）和 l2 penalty（l2惩罚）的回归
sgdr.fit(X_train, y_train)
y_pred = sgdr.predict(X_test) #预测值
print(sgdr.intercept_)
print(sgdr.coef_)
print ( "MSE: %.2f" %mean_squared_error(y_test, y_pred)) #MSE
print ( "R^2: %.2f" %r2_score(y_test, y_pred)) #R^2

SGDRegressor(alpha=0.01)

[0.52012837]
[-0.35901253  0.24656547  0.61170052  0.41274754  0.11719216  0.58443304]
MSE: 0.05
R^2: 0.91
